## This tests the standard RHF gradient components for cc-pVDZ water against psi4 scfgrad with cavity effects

We will turn the cavity on along z and we will update psi4's wavefunction object to use the density from the cqed-rhf calculation!

In [ ]:
from oo_cqed_rhf import CQEDRHFCalculator
import numpy as np
import psi4

In [ ]:
molecule h2 {
         O            0.000000000000     0.000000000000    -0.068516219320
         H            0.000000000000    -0.790689573744     0.543701060715
         H            0.000000000000     0.790689573744     0.543701060715
no_reorient
nocom
symmetry c1
}
set {
  basis sto-3g
  scf_type pk
  e_convergence 1e-12
  d_convergence 1e-12
}
# frequency: 2.0 eV
w = 0.07349864501573
# lambda: 0.05
# coupling g = 0.05 / sqrt(2*w)
g = 0.05 / np.sqrt(2.0 * w)
set {
  maxiter 500
  cavity_frequency         [0.0, 0.0, $w]
  cavity_coupling_strength [0.0, 0.0, $g]
}

In [ ]:
# Expected gradient contributions for Water in a STO-3G basis set with the following geometry:
h2o_string = """
         O            0.000000000000     0.000000000000    -0.068516219320
         H            0.000000000000    -0.790689573744     0.543701060715
         H            0.000000000000     0.790689573744     0.543701060715
no_reorient
nocom
symmetry c1
"""

w = 0.07349864501573
# lambda: 0.05
# coupling g = 0.05 / sqrt(2*w)
g = 0.05 / np.sqrt(2.0 * w)

# no cavity
lambda_vector = np.array([0, 0, g])
print(lambda_vector)
# options for H2O
psi4_options = {
    "basis": "sto-3g",
    "save_jk": True,
    "scf_type": "pk",
    "e_convergence": 1e-12,
    "d_convergence": 1e-12,
}





In [ ]:
calc = CQEDRHFCalculator(lambda_vector, h2o_string, psi4_options)
calc.calc_cqed_rhf_energy()

In [ ]:
calc.compute_canonical_gradients()

In [ ]:
# print the total gradient
print(calc.canonical_gradient)

In [ ]:
# use psi4 to compute the gradient
calc.calc_scf_gradient(qed_wfn=True)

In [ ]:
print(calc.scf_grad)

In [ ]:
print(F" My Gradient Matches Psi4's Gradient: {np.allclose(calc.scf_grad, calc.canonical_gradient, 1e-6, 1e-6)}")
error_norm = np.linalg.norm(calc.scf_grad-  calc.canonical_gradient)
print(F" Norm of difference is {error_norm}")